In [1]:
result = []
with open('breast-cancer-wisconsin.data.txt') as inputfile:
    for line in inputfile:
        if '?' in line:
            continue
        else:
            result.append(line.rstrip().split(','))

In [2]:
features = []
labels = []

In [3]:
print("Total number of Data\t",len(result))

Total number of Data	 683


In [4]:
print(result[0][0])

1000025


In [5]:
for i in range(len(result)):
    rs = [int(j) for j in result[i]]
    features.append(rs[1:10])
    labels.append(rs[10])

In [6]:
print(features[0:10])

[[5, 1, 1, 1, 2, 1, 3, 1, 1], [5, 4, 4, 5, 7, 10, 3, 2, 1], [3, 1, 1, 1, 2, 2, 3, 1, 1], [6, 8, 8, 1, 3, 4, 3, 7, 1], [4, 1, 1, 3, 2, 1, 3, 1, 1], [8, 10, 10, 8, 7, 10, 9, 7, 1], [1, 1, 1, 1, 2, 10, 3, 1, 1], [2, 1, 2, 1, 2, 1, 3, 1, 1], [2, 1, 1, 1, 2, 1, 1, 1, 5], [4, 2, 1, 1, 2, 1, 2, 1, 1]]


In [7]:
print(labels[0:10])

[2, 2, 2, 2, 2, 4, 2, 2, 2, 2]


In [20]:
labels = [0 if x==2 else x for x in labels]
labels = [1 if x==4 else x for x in labels]

In [21]:
print(labels[0:10])

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]


In [22]:
import numpy as np

In [23]:
np.shape(features)

(683, 9)

In [24]:
np.shape(labels)

(683,)

In [28]:
from keras.utils.np_utils import to_categorical   

labels = to_categorical(labels, num_classes=2)

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [30]:
print(labels[0:10])

[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [44]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train, Y_test = train_test_split(features, labels, test_size = 0.33)

In [46]:
print("shape of train data ", np.shape(X_train))
print("shape of test data", np.shape(Y_test))

shape of train data  (457, 9)
shape of test data (226, 2)


In [31]:
import tensorflow as tf

In [53]:
learning_rate = 0.001
training_epochs =  20
batch_size = 32
display_step = 1

In [62]:
n_hidden1 = 256
n_hidden2 = 512
n_hidden3 = 256
n_classes = 2
n_input = 9

In [63]:
X = tf.placeholder('float',[None,n_input])
Y = tf.placeholder('float',[None,n_classes])

In [66]:
weights = {
    'h1':tf.Variable(tf.random_normal([n_input,n_hidden1])),
    'h2':tf.Variable(tf.random_normal([n_hidden1,n_hidden2])),
    'h3':tf.Variable(tf.random_normal([n_hidden2,n_hidden3])),
    'out':tf.Variable(tf.random_normal([n_hidden3,n_classes]))
}

biases = {
    'b1':tf.Variable(tf.random_normal([n_hidden1])),
    'b2':tf.Variable(tf.random_normal([n_hidden2])),
    'b3':tf.Variable(tf.random_normal([n_hidden3])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

In [69]:
def multilayer_perceptron(x):
    layer_1 = tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_3 = tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    out_layer = tf.matmul(layer_3,weights['out'])+biases['out']
    return out_layer

In [70]:
logits = multilayer_perceptron(X)

In [71]:
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train_op = optimizer.minimize(loss_op)
init = tf.global_variables_initializer()

In [72]:
def get_next_batch(batch_size, features, labels):
    idx = np.arange(0, len(features))
    np.random.shuffle(idx)
    idx = idx[:batch_size]
    data = [features[i] for i in idx]
    labl = [labels[i] for i in idx]
    return np.asarray(data), np.asarray(labl)
    

In [73]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int((np.shape(X_train)[0]/batch_size))
        
        for i in range(total_batch):
            batch_x, batch_y = get_next_batch(batch_size, X_train, Y_train)
            _,c = sess.run([train_op, loss_op], feed_dict={X:batch_x, Y:batch_y})
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    
    pred = tf.nn.softmax(logits)  
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
                
        
        

Epoch: 0001 cost=46987.577933175
Epoch: 0002 cost=10734.254525321
Epoch: 0003 cost=5940.571105957
Epoch: 0004 cost=4529.269208636
Epoch: 0005 cost=4846.374585833
Epoch: 0006 cost=3386.833833967
Epoch: 0007 cost=2329.597255162
Epoch: 0008 cost=1579.664975848
Epoch: 0009 cost=2142.185793195
Epoch: 0010 cost=1690.824626378
Epoch: 0011 cost=1398.393345424
Epoch: 0012 cost=1120.690298898
Epoch: 0013 cost=852.983777727
Epoch: 0014 cost=1115.431108747
Epoch: 0015 cost=505.447014945
Epoch: 0016 cost=310.587127686
Epoch: 0017 cost=358.258178711
Epoch: 0018 cost=228.025573730
Epoch: 0019 cost=98.662384033
Epoch: 0020 cost=96.829952861
Epoch: 0021 cost=31.469142369
Epoch: 0022 cost=40.490971157
Epoch: 0023 cost=97.901951109
Epoch: 0024 cost=26.304761614
Epoch: 0025 cost=98.672197614
Epoch: 0026 cost=127.872752598
Epoch: 0027 cost=31.050545831
Epoch: 0028 cost=81.843418666
Epoch: 0029 cost=64.474905831
Epoch: 0030 cost=84.130972794
Epoch: 0031 cost=77.745837075
Epoch: 0032 cost=48.200435093
Epoch: